In [1]:
import json
from PIL import Image
import os
import numpy as np
import cv2
import tqdm


IMAGE_PATH = os.path.expanduser("LEVIR-MCI-dataset/images")
RAW_JSON_PATH = os.path.expanduser("LEVIR-MCI-dataset/LevirCCcaptions.json")

with open(RAW_JSON_PATH, "r") as f: 
    raw_json = json.load(f)
raw_captions = raw_json['images']
# raw_captions = [x for x in raw_captions if x['filename'] == 'test_001000.png']

In [151]:
def getChageMask(filepath, filename):
    changeMask = Image.open(os.path.join(IMAGE_PATH, filepath, "label", filename))
    changeMask = np.array(changeMask)
    return changeMask

def analyze_segmentation_mask(mask):
    # 目标颜色定义
    ROAD = [128, 128, 128]
    BUILDING = [255, 255, 255]
    BACKGROUND = [0, 0, 0]

    # 1) 判断掩码图中是否有目标
    unique_colors = np.unique(mask.reshape(-1, mask.shape[2]), axis=0)
    has_target = any((color != BACKGROUND).all() for color in unique_colors)

    # 2) 计算每个类别的目标数量
    def count_objects(color):
        binary_mask = cv2.inRange(mask, np.array(color), np.array(color))
        contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        return len(contours), contours

    road_count, road_contours = count_objects(ROAD)
    building_count, building_contours = count_objects(BUILDING)

    target_counts = {
        "road": road_count,
        "building": building_count
    }

    # 3) 生成每个目标的边界轮廓坐标，使用cv2.approxPolyDP减少顶点数量
    def simplify_contours(contours, epsilon=2, min_vertices=2, max_vertices=8):
        simplified_contours = []
        for contour in contours:
            # contour = cv2.convexHull(contour)
            # 动态调整epsilon值，直到顶点数在指定范围内
            epsilon_value = epsilon * cv2.arcLength(contour, True) / 100
            approx = cv2.approxPolyDP(contour, epsilon_value, True)
            
            # 增加epsilon值以减少顶点数，不超过最大顶点数
            while len(approx) > max_vertices and epsilon_value < cv2.arcLength(contour, True):
                epsilon_value += 0.1
                approx = cv2.approxPolyDP(contour, epsilon_value, True)
            
            simplified_contours.append(approx)
        return simplified_contours


    road_contours_simplified = simplify_contours(road_contours)
    building_contours_simplified = simplify_contours(building_contours)

    contours = {
        "road": [contour.reshape(-1,2).tolist() for contour in road_contours_simplified],
        "building": [contour.reshape(-1,2).tolist() for contour in building_contours_simplified]
    }

    return has_target, target_counts, contours, road_contours_simplified, building_contours_simplified

In [135]:
rslt_v2 = []
maybeError = []
for row in tqdm.tqdm(raw_captions):
    mask = getChageMask(row['filepath'], row['filename'])
    has_target, target_counts, contours, road_contours, building_contours = analyze_segmentation_mask(mask)
    _newRow = {
        "filepath": row['filepath'],
        "filename": row['filename'],
        "imageid": row['imgid'],
        "sentences": [x['raw'] for x in row["sentences"]],
        "changeflag": row["changeflag"],
        "changeflag2": int(has_target),
        "change_counts": target_counts,
        "contours": contours
        }
    # if has_target and contours['road']:
    #     assert(np.array(np.expand_dims(contours['road'][0], 1)) == road_contours[0]).all()
    rslt_v2.append(_newRow)

with open(os.path.expanduser("LEVIR-MCI-dataset/LevirCCcaptions-v3.json"), 'w') as f:
    json.dump(rslt_v2, f)

100%|██████████| 10077/10077 [09:32<00:00, 17.60it/s]


In [3]:
with open(os.path.expanduser("LEVIR-MCI-dataset/LevirCCcaptions-v2.json"), 'r') as f:
    rslt_v2 = json.load(f)

def process_numbers(obj):
    if isinstance(obj, list):
        return [process_numbers(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: process_numbers(value) for key, value in obj.items()}
    elif isinstance(obj, int):
        return round(obj / 256, 2)  # 将数字除以 256 并保留两位小数
    else:
        return obj

In [5]:
for row in rslt_v2:
    row['contours'] = process_numbers(row['contours'])

In [9]:
with open(os.path.expanduser("LEVIR-MCI-dataset/LevirCCcaptions-v2-f.json"), 'w') as f:
    json.dump(rslt_v2, f)

In [171]:
from matplotlib import pyplot as plt

def draw_contours_on_image(image_path, road_contours, building_contours):
    # 读取原图像
    image = cv2.imread(image_path)
    cv2.cvtColor(image, cv2.COLOR_BGR2RGB, image)

    # 颜色定义
    road_color = (0, 255, 0) # 绿色
    building_color = (0, 0, 255) # 蓝色

    road_contours = [np.array(x) for x in road_contours]
    building_contours = [np.array(x) for x in building_contours]

    # 在图像上绘制道路轮廓
    cv2.drawContours(image, road_contours, -1, road_color, 2)
    # 在图像上绘制建筑物轮廓
    cv2.drawContours(image, building_contours, -1, building_color, 2)

    return image

for row in rslt_v3:
    label_path = f"LEVIR-MCI-dataset/images/{row['filepath']}/label/{row['filename']}"
    rslt_path = f"LEVIR-MCI-dataset/images/{row['filepath']}/contour/{row['filename']}"
    if not os.path.exists(os.path.dirname(rslt_path)):
        os.makedirs(os.path.dirname(rslt_path))
    road_contours = row['contours']['road']
    building_contours = row['contours']['building']
    annotated_image = draw_contours_on_image(label_path, road_contours, building_contours)

    cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR, annotated_image)
    cv2.imwrite(rslt_path, annotated_image)
